In [1]:
import json
import numpy as np
import pandas as pd
import psycopg2
import os

from collections import Counter
from datetime import datetime, timedelta
from dateutil.tz import tzlocal 
from pathlib import Path
from tqdm import tqdm

## Analyse data

In [2]:
[p for p in kharkiv_folder_path.iterdir()]

[WindowsPath('../../../pet_project/kharkiv/new 1.txt'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_01_FEB_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_02_FEB_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_03_FEB_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_25_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_26_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_27_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_28_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_29_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_30_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_31_JAN_2023')]

In [3]:
[p for p in Path("../../data/local/jsons/kharkiv").iterdir()]

[WindowsPath('../../data/local/jsons/kharkiv/trans_data_03_FEB_2023'),
 WindowsPath('../../data/local/jsons/kharkiv/trans_data_04_FEB_2023'),
 WindowsPath('../../data/local/jsons/kharkiv/trans_data_05_FEB_2023')]

In [2]:
kharkiv_folder_path = Path("../../../pet_project/kharkiv")
kharkiv_folder_list = [p for p in kharkiv_folder_path.iterdir() if "trans_data_" in p.name]
kharkiv_folders_list = sorted(kharkiv_folder_list,
                        key=lambda p: datetime.strptime(p.name[11:], '%d_%b_%Y'))
kharkiv_folders_list

[WindowsPath('../../../pet_project/kharkiv/trans_data_25_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_26_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_27_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_28_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_29_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_30_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_31_JAN_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_01_FEB_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_02_FEB_2023'),
 WindowsPath('../../../pet_project/kharkiv/trans_data_03_FEB_2023')]

In [3]:
from concurrent.futures import ThreadPoolExecutor

def load_responses(resp_path):
    resp_tm = datetime.strptime(resp_path.name[6:-5], '%Y-%m-%d %H;%M;%S')
    resp_tm_str = resp_tm.replace(tzinfo=tzlocal()).isoformat()
    
    with open(resp_path) as f:
        resp = json.load(f)
        if ("rows" in resp) and (resp["rows"]): 
            return [row + [resp['timestamp'], resp_tm_str] 
                    for row in resp['rows']]
        else: 
            return []

def accumulate_responses_from_folder(folder_path):
    file_path_list = list(folder_path.iterdir())
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(tqdm(executor.map(load_responses, file_path_list),
                            total=len(file_path_list), mininterval=2, leave=False, 
                            desc="Accumulate responses from folder")
                          )
        resp_list = []
        for resp_ in results: 
            resp_list += resp_
        print(f"Accumulate responses from folder\n"\
              f"\t{len(resp_list)} / {len(resp_list)} [avg={len(resp_list)/len(resp_list):.2f}]")
    
    
    columns = ['imei', 'lat', 'lng', 'speed', 'gps_datetime_origin', 'orientation', 'route_name', 
               'route_type', 'vehcle_id', 'dd', 'gpstime', 'response_datetime',] 

    return pd.DataFrame(resp_list, columns=columns)
    
def clear_data(in_df):
    unique_data = []
    
    imei_list = in_df['imei'].value_counts().index
    imei_tqdm = tqdm(
        imei_list, total=len(imei_list), mininterval=2, leave=False, desc="Clear data"
    )
    
    for imei in imei_tqdm: 
        row_data = in_df[ in_df['imei'] == imei].values.tolist()
        result = [row_data[0]]
        for row0, row1 in zip(row_data[:-1], row_data[1:]):
            if row0[1:9] != row1[1:9]: 
                result += [row1]
        unique_data += result

    df_unique = pd.DataFrame(unique_data, columns=in_df.columns)
    print(f"Clear data\n\t{len(in_df)} / {len(df_unique)} [avg {len(in_df) / len(df_unique):.02f}]")
    return df_unique

In [4]:
import PTETA
import importlib
# importlib.reload(PTETA.utils.transport.kharkiv)

try: del KharkivTransportAVLData
except: pass 
finally: from PTETA.utils.transport.kharkiv.KharkivTransportAVLData import KharkivTransportAVLData
    
try: del KharkivTransportOperator
except: pass 
finally: from PTETA.utils.transport.kharkiv.KharkivTransportOperator import KharkivTransportOperator
    
try: del KharkivTransportRoute
except: pass 
finally: from PTETA.utils.transport.kharkiv.KharkivTransportRoute import KharkivTransportRoute
    
try: del KharkivTransportVehicle
except: pass 
finally: from PTETA.utils.transport.kharkiv.KharkivTransportVehicle import KharkivTransportVehicle

try: del TransGPSCVMonitor
except: pass 
finally: from aws.src.TransGPSCVMonitor import TransGPSCVMonitor

In [12]:
BATCH_SIZE = 3_000
connection_config= dict({
    'host': os.environ['RDS_HOSTNAME'],
    'database': "pteta_db",
    'user': "postgres",
    'password': os.environ['RDS_PTETA_DB_PASSWORD']
})

monitor = TransGPSCVMonitor(connection_config=connection_config, data_model="kharkiv")

# file_path_list =list(Path("../data/local/tables/").iterdir())
file_path_list =list(Path("D:/projects/pet_project/tables").iterdir())
file_path_list = sorted(file_path_list, 
                        key=lambda p: datetime.strptime(p.name[9:-4], '%d_%b_%Y') )

for folder_path in tqdm(kharkiv_folders_list[3:]):
    
    df = accumulate_responses_from_folder(folder_path)
    df_u = clear_data(df)
    
#     df_sum = pd.read_csv(df_path, encoding='utf-8', low_memory=False)
        
#     if "response_datetime" not in df_sum: 
#         df_sum['response_datetime'] = None

#     if not df_sum['imei'].dtype == 'O':
#         df_sum['imei'] = df_sum['imei'].astype(str, copy=True)

#     if not df_sum['busNumber'].dtype == 'O':
#         df_sum['busNumber'] = df_sum['busNumber'].astype(str, copy=True)
        
    
    df_cur = df_u[:]

    batch_tqdm = tqdm(df_cur.groupby(np.arange(len(df_cur)) // BATCH_SIZE), 
                     mininterval=10)
    for batch_number, batch_df in batch_tqdm:
        batch_df = batch_df.where(pd.notnull(batch_df), None)
        monitor.write_to_db(batch_df.to_dict('records'))

Accumulate responses from folder:  94%|█████████▍| 3127/3334 [00:10<00:00, 301.43it/s]
                                                                                      

Accumulate responses from folder
	118883 / 118883 [avg=1.00]



Clear data:   0%|          | 0/233 [00:00<?, ?it/s]
                                                   
  0%|          | 0/4 [00:00<?, ?it/s]

Clear data
	118883 / 9609 [avg 12.37]
There are 3 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportVehicle.KharkivTransportVehicle'> to inserted in DB
There are 2 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportVehicle.KharkivTransportVehicle'> to inserted in DB
There are 1 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportVehicle.KharkivTransportVehicle'> to inserted in DB


Accumulate responses from folder:  96%|█████████▌| 16517/17280 [00:54<00:02, 363.12it/s]
                                                                                        

...

Clear data
	1936194 / 183442 [avg 10.55]



 85%|████████▌ | 53/62 [00:33<00:05,  1.56it/s]

There are 1 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportVehicle.KharkivTransportVehicle'> to inserted in DB


 86%|████████▌ | 6/7 [11:03<01:50, 110.65s/it]                         


ZeroDivisionError: division by zero

In [6]:
monitor.objects_unique

{PTETA.utils.transport.kharkiv.KharkivTransportOperator.KharkivTransportOperator: {KharkivTransportOperator(id=-1, name='UNKNOWN')},
 PTETA.utils.transport.kharkiv.KharkivTransportRoute.KharkivTransportRoute: {KharkivTransportRoute(id=-1),
  KharkivTransportRoute(id=1),
  KharkivTransportRoute(id=10),
  KharkivTransportRoute(id=11),
  KharkivTransportRoute(id=12),
  KharkivTransportRoute(id=13),
  KharkivTransportRoute(id=14),
  KharkivTransportRoute(id=15),
  KharkivTransportRoute(id=16),
  KharkivTransportRoute(id=17),
  KharkivTransportRoute(id=18),
  KharkivTransportRoute(id=19),
  KharkivTransportRoute(id=2),
  KharkivTransportRoute(id=20),
  KharkivTransportRoute(id=21),
  KharkivTransportRoute(id=22),
  KharkivTransportRoute(id=23),
  KharkivTransportRoute(id=24),
  KharkivTransportRoute(id=25),
  KharkivTransportRoute(id=5),
  KharkivTransportRoute(id=6),
  KharkivTransportRoute(id=7),
  KharkivTransportRoute(id=8),
  KharkivTransportRoute(id=9)},
 PTETA.utils.transport.kharkiv

In [7]:
operator_list, route_list, vehicle_list, avl_data_list = monitor.decompose_response(batch_df.to_dict('records'))

for obj_list in [operator_list, route_list, vehicle_list]:
    new_obj = monitor.get_new_objs(obj_list)
    if new_obj:
        print(f"There are {len(new_obj)} new {new_obj[0].__class__} to inserted in DB")
        monitor.update_db(new_obj)

for i, (vehicle, route) in enumerate(zip(vehicle_list, route_list)):
    avl_data_list[i].vehicle_id = monitor.vehicle_to_id[vehicle]
    avl_data_list[i].route_id = monitor.route_to_id[route]


There are 1 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportRoute.KharkivTransportRoute'> to inserted in DB


KeyError: KharkivTransportRoute(id=None)

In [8]:
i, (vehicle, route)

(732,
 (KharkivTransportVehicle(id=None, imei='353976014249548', name='None', owner_id=-1),
  KharkivTransportRoute(id=None)))

In [9]:
route.id, route.name, route.type

(None, '27', 1)

In [ ]:
route.__

In [10]:
[(r.id, r.name, r.type) for r in monitor.objects_unique[route.__class__]]

[(8, '8', 1),
 (25, '28', 1),
 (9, '208', 3),
 (16, '3', 2),
 (23, '75', 3),
 (6, '204', 3),
 (22, '68', 3),
 (15, '206', 3),
 (14, '212', 3),
 (19, '51', 2),
 (-1, 'UNKNOWN', -1),
 (17, '1', 2),
 (21, '20', 1),
 (18, '6', 2),
 (7, '45', 2),
 (12, '35', 2),
 (11, '46', 2),
 (1, '52', 2),
 (2, '55', 2),
 (5, '24', 2),
 (20, '13', 2),
 (13, '72', 3),
 (10, '34', 2),
 (24, '27', 2)]

In [7]:
#     df_cur = df_u[:]

batch_tqdm = tqdm(df_cur.groupby(np.arange(len(df_cur)) // BATCH_SIZE), 
                  miniters=40)
for batch_number, batch_df in batch_tqdm:
    batch_df = batch_df.where(pd.notnull(batch_df), None)
    monitor.write_to_db(batch_df.to_dict('records'))

  0%|          | 0/124 [00:00<?, ?it/s]

There are 2 new <class 'PTETA.utils.transport.kharkiv.KharkivTransportRoute.KharkivTransportRoute'> to inserted in DB
Error raised while select <class 'PTETA.utils.transport.kharkiv.KharkivTransportRoute.KharkivTransportRoute'> '[KharkivTransportRoute(id=None, name='52', type=2), KharkivTransportRoute(id=None, name='55', type=2)]'


SyntaxError: syntax error at or near "AND"
LINE 1: SELECT "name", "type" FROM kharkiv.route  WHERE ( AND "name"...
                                                          ^


## TransportRoute

In [7]:
TransportRoute.__insert_columns__().replace('"', '').split(', ')[1:]

['route_name', 'route_colour']

In [28]:
cols = ["routeId", 'routeName', 'routeColour']
route_list = [TransportRoute.from_response_row(row) for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(route_list), route_list[:3]
TransportRoute.insert_many_in_table(conn, route_list)

In [32]:
TransportRoute.get_table(conn)

[TransportRoute(id=37, name='T', colour='coral'),
 TransportRoute(id=31, name='6/6a', colour='deeppink'),
 TransportRoute(id=21, name='38', colour='deeppink'),
 TransportRoute(id=41, name='10A', colour='black'),
 TransportRoute(id=20, name='A', colour='navy'),
 TransportRoute(id=42, name='39', colour='coral'),
 TransportRoute(id=23, name='19', colour='teal'),
 TransportRoute(id=19, name='10', colour='black'),
 TransportRoute(id=11, name='9', colour='magenta'),
 TransportRoute(id=2, name='11', colour='green'),
 TransportRoute(id=6, name='4', colour='magenta'),
 TransportRoute(id=27, name='3/3a', colour='green'),
 TransportRoute(id=16, name='2', colour='green'),
 TransportRoute(id=4, name='5', colour='orange'),
 TransportRoute(id=3, name='12', colour='blue'),
 TransportRoute(id=12, name='20', colour='maroon'),
 TransportRoute(id=7, name='6', colour='sienna'),
 TransportRoute(id=10, name='34', colour='navy'),
 TransportRoute(id=45, name='1', colour='navy'),
 TransportRoute(id=9, name='27'

In [13]:
# df_sum[TransportRoute.__insert_columns__().replace('"', '').split(', ')[1:]].value_counts()

In [27]:
conn.rollback()

In [45]:
SQL_big_req = " ".join([create_sql_req(t) for t in trans_vehicle_list])

In [29]:
route_list = TransportRoute.get_table(conn)[:]
len(route_list)

31

In [30]:
for r in route_list[::2]: 
    r.id += 1
TransportRoute.are_in_table(conn, route_list)

[False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False]

In [31]:
TransportRoute.are_in_table(conn, route_list)

[False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False]

In [14]:
for r in route_list: 
    r.name += "_"
    r.id += 100

In [16]:
# sql = f"""INSERT INTO pteta.route("id", "routeName", "routeColour") VALUES """ + \
#               ", ".join([f"""({obj.id}, '{obj.name}', '{obj.colour}')"""
#                         for obj in route_list]) + ";"
# sql

In [17]:
# route_list

In [18]:
TransportRoute.insert_many_in_table(conn, route_list)

## Test TransportVehicle

In [33]:
TransportVehicle.__insert_columns__()

'"imei", "name", "bus_number", "remark", "perev_id"'

In [45]:
cols = ["imei", "name", "busNumber", "remark", "perevId"]
vehicle_list = [TransportVehicle.from_response_row(row) for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(vehicle_list), vehicle_list[:3]
TransportVehicle.insert_many_in_table(conn, vehicle_list)

In [47]:
vehicle_list = TransportVehicle.get_table(conn)
len(vehicle_list)

99

In [48]:
TransportVehicle.are_in_table(conn, vehicle_list[:5])

[True, True, True, True, True]

In [49]:
vehicle_list[5].is_in_table(conn)

True

In [50]:
obj = vehicle_list[0]
obj.imei += "-"
obj.is_in_table(conn)

False

In [51]:
obj.insert_in_table(conn)

In [52]:
for v in vehicle_list[:10]: 
    v.imei += '_'

In [15]:
TransportVehicle.insert_many_in_table(conn, vehicle_list[:10])

In [31]:
obj.insert_in_table(conn)
obj.is_in_table(conn)

In [14]:
vehicle_list[0].id = None
print(vehicle_list[0])
vehicle_list[0].update_id_from_table(conn)
print(vehicle_list[0])

TransportVehicle(id=None, imei='355227045600830', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37)
TransportVehicle(id=1, imei='355227045600830', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37)


In [34]:
for v in vehicle_list[:10]: 
    v.imei += '__'

vehicle_list[:13]

[TransportVehicle(id=222, imei='355227045600830-__', name='A178', busNumber='310', remark='Тролейбус 310 DNSNK', perevId=6, routeId=37),
 TransportVehicle(id=2, imei='355227046451662__', name='H76', busNumber='350', remark='Тролейбус 350 DNSNTNK', perevId=6, routeId=31),
 TransportVehicle(id=3, imei='355227045369527__', name='A6', busNumber='3557', remark='3557 DNSNK ', perevId=12, routeId=21),
 TransportVehicle(id=4, imei='355227045540176__', name='A83', busNumber='5150', remark=' 5150 DNS', perevId=7, routeId=41),
 TransportVehicle(id=5, imei='355227046453387__', name='H75', busNumber='3627', remark='3627 DNSNTNK', perevId=1, routeId=20),
 TransportVehicle(id=6, imei='355228042084283__', name='A207', busNumber='1032', remark='1032 DNSNK', perevId=13, routeId=42),
 TransportVehicle(id=7, imei='355227045371655__', name='A1', busNumber='6513', remark='6513 DNSNK', perevId=7, routeId=23),
 TransportVehicle(id=8, imei='355227046451407__', name='H68', busNumber='0855', remark='0855 DNSNTNK

## TransportOperator

In [41]:
operator_list = TransportOperator.get_table(conn)
len(operator_list)

6

In [37]:
TransportOperator.__insert_columns__()

'id, "perev_name"'

In [40]:
cols = ['perevId', "perevName"]
operator_list = [TransportOperator.from_response_row(row) 
              for row in df_sum[cols].drop_duplicates().to_dict('records')]
len(operator_list), operator_list[:3]
TransportOperator.insert_many_in_table(conn, operator_list)

In [42]:
TransportOperator.are_in_table(conn, operator_list[:])

[True, True, True, True, True, True]

In [43]:
operator_list[0].is_in_table(conn)

True

In [44]:
obj = operator_list[0]
obj.id += 100
obj.name = "-_-" + obj.name
obj.is_in_table(conn)

False

In [14]:
obj.insert_in_table(conn)

In [15]:
for op in operator_list: 
    op.id += 100
    op.name = "-_-" + op.name
    
TransportOperator.insert_many_in_table(conn, operator_list)

## TransportAVLData

In [9]:
avl_data_list = TransportAVLData.get_table(conn)

In [11]:
# avl_data_list

In [14]:
%%timeit
TransportAVLData.are_in_table(conn, avl_data_list)

57.4 ms ± 3.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
avl_data_list[0].is_in_table(conn)

42.1 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
